<a href="https://colab.research.google.com/github/ipeirotis-org/datasets/blob/main/Cybersyn_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q snowflake


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:

fred_ids =  (
    session.table("FINANCIAL_FRED_VARIABLE_SERIES_ID_CROSSWALK")
    .select("VARIABLE", "SERIES_ID")
)

macro_data =  (
    session.table("FINANCIAL_FRED_TIMESERIES")
    .select('VARIABLE', 'DATE', 'VALUE')
)

fred_data_ids = (
    fred_ids
    .join(macro_data, fred_ids.col("VARIABLE") == macro_data.col("VARIABLE"))
    .select(fred_ids["VARIABLE"].as_("VARIABLE"), "SERIES_ID", "DATE", "VALUE")
)

cpi_pcepi_data = (
    fred_data_ids
    .filter(fred_data_ids['SERIES_ID'].in_("PCEPI", "CPIAUCSL"))
    .sort(col("DATE"))
)

In [ ]:
import json
from snowflake.snowpark.session import Session

connection_parameters = {
  "account" : "cfcxbpg-ct14445",
  "user" : "ipeirotis",
  "password" : "QYK.uye.npr.rgv1fru",
  "database" : "cybersyn_financial__economic_essentials",
  "schema" : "cybersyn"
}
session = Session.builder.configs(connection_parameters).create()


sql_query = """
WITH fred_ids AS (
    SELECT VARIABLE, SERIES_ID
    FROM cybersyn_financial__economic_essentials.cybersyn.FINANCIAL_FRED_VARIABLE_SERIES_ID_CROSSWALK
),
macro_data AS (
    SELECT VARIABLE, DATE, VALUE
    FROM cybersyn_financial__economic_essentials.cybersyn.FINANCIAL_FRED_TIMESERIES
),
fred_data_ids AS (
    SELECT
        f.VARIABLE,
        f.SERIES_ID,
        m.DATE,
        m.VALUE
    FROM fred_ids f
    JOIN macro_data m ON f.VARIABLE = m.VARIABLE
)
SELECT
    VARIABLE,
    SERIES_ID,
    DATE,
    VALUE
FROM fred_data_ids
WHERE SERIES_ID IN ('PCEPI', 'CPIAUCSL')
ORDER BY DATE;
"""

result = session.sql(sql_query).collect()
df = pd.DataFrame(result, columns=["VARIABLE", "SERIES_ID", "DATE", "VALUE"])


In [ ]:
plot = df.pivot_table(
    index='DATE',
    columns='SERIES_ID',
    values='VALUE'
).plot(
    linestyle='--',
    linewidth=2,
    markersize=10,
    figsize=(18, 6), grid=True,
    title='Consumer Price Index (CPI) vs. Personal Consumption Expenditures (PCEPI)'
)

plot.set_xlabel('Date')
plot.set_ylabel('CPI')

pass